In [1]:
#import calendar
import datetime
#import dateutil.parser
import gpxpy 
import gpxpy.gpx
import os
import pandas as pd
import pickle
import pendulum
import pydarksky
import numpy as np
import requests
import sys
import time
import timeit

from glob import glob
from pathlib import Path

#from xml.dom import minidom
#from gpx_csv_converter import Converter

In [2]:
# Setup directory for local libraty Import in notebook
current_dir = get_ipython().getoutput('pwd')
module_path = Path(current_dir[0]).parent

if module_path not in sys.path:
    sys.path.append(str(module_path))

# importy Modified gpx_csv_converter CLASS
from gpx_csv_convert import Converter

In [3]:
# Path to data - files operations

PATH_TO_DATA = os.path.abspath(os.curdir)
path_to_files = 'gpx\\running\\*'

# File operations
file_quant = len(glob(path_to_files))
file_names = [file for file in glob(path_to_files)]
file_length = len(file_names)

#files_gpx = os.listdir('gpx')
#files_gpx = [('gpx\\' + file) for file in os.listdir('gpx')]
files_csv = [ ('csv' + path_to_files[3:-1] + file[:len(file) - 4] + '.csv') for file in os.listdir(path_to_files[:-2])]

print(' path from gpx:\n',file_names)
print('\n path to csv: \n',files_csv)

 path from gpx:
 ['gpx\\running\\Snow_corridor_Street_porridge.gpx', 'gpx\\running\\Snow_waist_Water_under_snow_Hard_snow.gpx', 'gpx\\running\\Some_intervals_in_snow_Frosty_roads.gpx']

 path to csv: 
 ['csv\\running\\Snow_corridor_Street_porridge.csv', 'csv\\running\\Snow_waist_Water_under_snow_Hard_snow.csv', 'csv\\running\\Some_intervals_in_snow_Frosty_roads.csv']


In [4]:
%%time

# OPEN GPS file for XML parse (modified libruary)

# gpx_file_xml = []

# convert gpx -> csv
for ind, (csv, file) in enumerate(zip(files_csv, file_names)):
    
    start_time = timeit.default_timer()
    
    gpx_file = open( file, 'r' )
    gpx_file_str = gpx_file.read()
    
    # parse gps by xml tags
    Converter(gpx_file_str, csv[:len(csv) - 4])
   
    gpx_file.close()
    
    end_time = timeit.default_timer() - start_time
    
    print(f'file: {ind + 1} -> {file}  time: {end_time:.0f} s \n')

file: 1 -> gpx\running\Snow_corridor_Street_porridge.gpx  time: 4 s 

file: 2 -> gpx\running\Snow_waist_Water_under_snow_Hard_snow.gpx  time: 5 s 

file: 3 -> gpx\running\Some_intervals_in_snow_Frosty_roads.gpx  time: 4 s 

Wall time: 13.1 s


In [5]:
# Import converted csv files
user_data = [pd.read_csv(os.path.join(PATH_TO_DATA,file)) for file in files_csv]

# Convert time and date to timestamp format (All files)
for data in user_data:
    # add datatime type
    data.insert(2,'data_time',(data['date'] + ' ' + data['time'])\
        .apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" )))
    # drop str time adn dates
    data.drop(['date','time'], axis=1, inplace=True)
    # add sport type (target)
    data['sport_type'] = path_to_files[4:-2]
    data['id'] = 'philka_ua'
    data['athlete'] = 'amateur'
    
user_data[0].head(10)

,data_time,latitude,longitude,elevation,heart_rate,cadence,temperature,sport_type,id,athlete
0,2019-01-28 13:56:10,49.409331,32.097608,90.4,97,61.0,NaN,running,philka_ua,amateur
1,2019-01-28 13:56:10,49.409345,32.097593,90.4,98,61.0,NaN,running,philka_ua,amateur
2,2019-01-28 13:56:11,49.409358,32.097570,90.2,98,61.0,NaN,running,philka_ua,amateur
3,2019-01-28 13:56:12,49.409369,32.097546,90.2,97,61.0,NaN,running,philka_ua,amateur
4,2019-01-28 13:56:13,49.409380,32.097521,90.2,97,61.0,NaN,running,philka_ua,amateur
5,2019-01-28 13:56:14,49.409392,32.097495,89.8,97,85.0,NaN,running,philka_ua,amateur
6,2019-01-28 13:56:15,49.409405,32.097469,89.8,97,85.0,NaN,running,philka_ua,amateur
7,2019-01-28 13:56:16,49.409411,32.097439,89.3,98,85.0,NaN,running,philka_ua,amateur
8,2019-01-28 13:56:17,49.409413,32.097411,89.1,98,84.0,NaN,running,philka_ua,amateur
9,2019-01-28 13:56:18,49.409405,32.097384,89.3,98,83.0,NaN,running,philka_ua,amateur
